### Multiword Expressions


Wydzielaliśmy do tej pory słowa w izolacji, choć wiemy, że występują one grupami. Jak stwierdzić, czy to, co obserwujemy w tekście to przypadkowa konstrukcja (nazwy rzeczy wymieniane jednym tchem, występujące obok siebie takie a nie inne słowa) czy może stałe połączenie wyrazowe, które możemy badać jako zjawisko (na wiele sposobów!) lub wykluczyć z korpusu, który poddajemy badaniom. 

Trwałe połączenia wyrazowe to frazy, które znamy jako przysłowia, powiedzonka, nazwy, złożenia. Więcej o nich i ich istocie można poczytać tu: https://en.wikipedia.org/wiki/Multiword_expression. Warto jednak zaznaczyć, że koncepcja jednostki wielowyrazowej ulega modyfikacjom w zależności od obranego paradygmatu badawczego! Dobrze jest przemyśleć sobie, co rozumiemy przez tę konstrukcję i konsekwentnie stosować się do tej wizji w naszych badaniach.

Przedmiotem tego laboratorium jest koncept samego poszukiwania jednostek wielowyrazowych, ich opisu gramatycznego, zestawienia częstych z rzadkimi wystąpieniami w korpusie.

W ramach wykrywania bigramów, będziemy kolejno:
1. Pobierać i tokenizować korpus FIQA.
2. Obliczać liczbę wystąpień złożeń dwóch elementów w tekście. Dla znanego przez nas zdania: The quick brown fox jumped over lazy dog rozłożenie takich dwuelementowych segmentów wygląda następująco:
"the quick": 1
"quick brown": 1
"brown fox": 1
...
"dog .": 1
3. Wyselekcjonujemy te segmenty, które tworzone są tylko przez słowa (ciekawość badacza: dlaczego teraz? dlaczego działamy w taki sposób?)
4. Użyjemy miary Pointwise Mutual Informatio (PMI), by obliczyć siłę powiązań pomiędzy elementami tworzącymi segment. PMI (https://en.wikipedia.org/wiki/Pointwise_mutual_information) to tylko jeden ze sposobów na otrzymanie takiej informacji, inne opisano tu: https://onlinelibrary.wiley.com/doi/full/10.1111/lang.12225.
5. Znajdziemy najczęstsze segmenty występujące w tekście. Określimy też próg przypadkowych połączeń wyrazów jako 5 wystąpień, nie będzie nas interesowało nic, co występuje 5 razy i rzadziej.
6. Spojrzymy na cały problem z punktu widzenia składni, a więc oznaczymy segmenty, przypisując im interpretację morfologiczną, policzymy, które złożenia są najczęstsze (ciekawość badacza: jak najczęstrze kategorie morfologiczne mają się w porównaniu do naszych najczęstszych słów z wcześniejszych punktów?

Zaczynamy od przygotowania środowiska.

In [1]:
import string
from collections import Counter, defaultdict
from itertools import chain

import numpy as np
import spacy
from datasets import load_dataset
from spacy.lang.pl import Polish

##!pip install ipywidgets

Użyte moduły:
- `string` https://docs.python.org/3/library/string.html
- `collections` https://docs.python.org/3/library/collections.html
- `itertools` https://docs.python.org/3/library/itertools.html

## Zadanie 1

Pobieramy korpus, tokenizujemy go, używając do tego znanych już narzędzi i zasobów.

In [2]:
dataset = load_dataset('clarin-knext/fiqa-pl', 'corpus')['corpus']['text']
dataset = ' '.join(dataset)

nlp = Polish()
tokenizer = nlp.tokenizer

tokens = tokenizer(dataset)
tokens = list(map(lambda t: t.text.lower(), tokens))
tokens[:10]

Found cached dataset fiqa-pl (C:/Users/AGH/.cache/huggingface/datasets/clarin-knext___fiqa-pl/corpus/0.0.0/bada00640881ee3fd04c3b88df9edd435616d17e0a46faf05e63063858742140)


  0%|          | 0/1 [00:00<?, ?it/s]

['nie', 'mówię', ',', 'że', 'nie', 'podoba', 'mi', 'się', 'też', 'pomysł']

## Zadanie 2-3

Obliczamy bigramy:
(tu sprytne wyjaśnienie: https://www.exploredatabase.com/2020/04/bigram-probability-estimation-of-word-sequence-example.html)

In [3]:
bigrams = [a + ' ' + b for a, b in zip(tokens, tokens[1:])]
bigrams = list(filter(lambda x: len(x.split(' ')) == 2, bigrams))
bigrams_counter = Counter(bigrams)

for i, (key, val) in zip(range(1, 50), bigrams_counter.items()):
    print(f'{i}. {key}: {val}')

1. nie mówię: 276
2. mówię ,: 429
3. , że: 86094
4. że nie: 5014
5. nie podoba: 164
6. podoba mi: 234
7. mi się: 1398
8. się też: 98
9. też pomysł: 3
10. pomysł szkolenia: 1
11. szkolenia w: 7
12. w miejscu: 210
13. miejscu pracy: 69
14. pracy ,: 1638
15. , ale: 32549
16. ale nie: 4242
17. nie możesz: 2535
18. możesz oczekiwać: 55
19. oczekiwać ,: 246
20. że firma: 517
21. firma to: 28
22. to zrobi: 86
23. zrobi .: 59
24. . szkolenie: 13
25. szkolenie pracowników: 2
26. pracowników to: 10
27. to nie: 4412
28. nie ich: 45
29. ich praca: 27
30. praca –: 3
31. – oni: 3
32. oni tworzą: 2
33. tworzą oprogramowanie: 1
34. oprogramowanie .: 33
35. . być: 693
36. być może: 2075
37. może systemy: 1
38. systemy edukacyjne: 1
39. edukacyjne w: 2
40. w stanach: 953
41. stanach zjednoczonych: 935
42. zjednoczonych (: 37
43. ( lub: 2155
44. lub ich: 76
45. ich studenci: 1
46. studenci ): 4
47. ) powinny: 15
48. powinny trochę: 2
49. trochę martwić: 2


I filtrujemy bigramy, aby zachować tylko takie, które zawierają wyrazy składające się wyłącznie z liter polskiego alfabetu:

In [4]:
def discard_elements(iterable, filter_fn):
    discard = []
    
    for element in iterable:
        if filter_fn(element):
            discard.append(element)
            continue
    
    for d in discard:
        del iterable[d]


all_chars = string.ascii_letters + 'ęóąśłżźćńĘÓĄŚŁŻŹĆŃ '
filter_not_letters = lambda element: any([c not in all_chars for c in element])

discard_elements(bigrams_counter, filter_not_letters)

for i, (key, val) in zip(range(1, 50), bigrams_counter.items()):
    print(f'{i}. {key}: {val}')

1. nie mówię: 276
2. że nie: 5014
3. nie podoba: 164
4. podoba mi: 234
5. mi się: 1398
6. się też: 98
7. też pomysł: 3
8. pomysł szkolenia: 1
9. szkolenia w: 7
10. w miejscu: 210
11. miejscu pracy: 69
12. ale nie: 4242
13. nie możesz: 2535
14. możesz oczekiwać: 55
15. że firma: 517
16. firma to: 28
17. to zrobi: 86
18. szkolenie pracowników: 2
19. pracowników to: 10
20. to nie: 4412
21. nie ich: 45
22. ich praca: 27
23. oni tworzą: 2
24. tworzą oprogramowanie: 1
25. być może: 2075
26. może systemy: 1
27. systemy edukacyjne: 1
28. edukacyjne w: 2
29. w stanach: 953
30. stanach zjednoczonych: 935
31. lub ich: 76
32. ich studenci: 1
33. powinny trochę: 2
34. trochę martwić: 2
35. martwić się: 112
36. się o: 1728
37. o zdobycie: 10
38. zdobycie umiejętności: 1
39. umiejętności rynkowych: 8
40. rynkowych w: 13
41. w zamian: 465
42. zamian za: 333
43. za ich: 186
44. ich ogromne: 2
45. ogromne inwestycje: 4
46. inwestycje w: 185
47. w edukację: 21
48. zamiast wychodzić: 3
49. wychodzić z: 20

## Zadanie 4

Używamy PMI, żeby stwierdzić, jakie jest prawdopodobieństwo współwystępowania słów z bigramu w korpusie - PMI przybliża siłę powiązania słów. Im wyższy wynik, tym mocniej powiązane są słowa.

In [5]:
tokens_counter = Counter(tokens)


def pmi(bigram):
    x, y = bigram.split(' ')
    p_x = tokens_counter[x] / len(tokens_counter)
    p_y = tokens_counter[y] / len(tokens_counter)
    p_xy = bigrams_counter[bigram] / len(bigrams_counter)

    if p_x * p_y == 0 or p_xy == 0:
        return 0
    else:
        return np.log2(p_xy / (p_x * p_y))


bigrams_pmi = [(pmi(bigram), bigram) for bigram in bigrams_counter]
bigrams_pmi[:50]

[(-4.416926295451629, 'nie mówię'),
 (-6.757987787011004, 'że nie'),
 (-4.208622682465007, 'nie podoba'),
 (0.8842670497309287, 'podoba mi'),
 (-3.9519473098108984, 'mi się'),
 (-7.324130378143073, 'się też'),
 (-6.423896465035036, 'też pomysł'),
 (-3.1341487758968243, 'pomysł szkolenia'),
 (-7.28737177449006, 'szkolenia w'),
 (-5.474170594496768, 'w miejscu'),
 (-2.491980962135719, 'miejscu pracy'),
 (-5.8753647825451685, 'ale nie'),
 (-5.790793503182507, 'nie możesz'),
 (-2.9420546603597835, 'możesz oczekiwać'),
 (-5.648675281555828, 'że firma'),
 (-10.226940957378476, 'firma to'),
 (-4.735397496939331, 'to zrobi'),
 (-2.45197247245145, 'szkolenie pracowników'),
 (-10.204730973533314, 'pracowników to'),
 (-7.314117119818877, 'to nie'),
 (-10.930620257899763, 'nie ich'),
 (-5.040290601186146, 'ich praca'),
 (-2.654767456453104, 'oni tworzą'),
 (-2.220646926344994, 'tworzą oprogramowanie'),
 (-3.488812977099607, 'być może'),
 (-8.573072285148388, 'może systemy'),
 (0.23089461624169236,

## Zadanie 5

Bezpośrednie zastosowanie PMI na korpusie zwraca dość charakterystyczne bigramy, które wyglądają, jakby występowały w korpusie bardzo rzadko (zarówno bigramy, jak i tworzące je tokeny) i dlatego znalazły się na początku rankingu (przez bardzo małe prawdopodobieństwa $p_x, p_y$).

In [6]:
sorted(bigrams_pmi, reverse=True)[:50]

[(14.412745683990813, 'żelem silikonowym'),
 (14.412745683990813, 'żarówkom żarowym'),
 (14.412745683990813, 'świnkami morskimi'),
 (14.412745683990813, 'światłami wystawowymi'),
 (14.412745683990813, 'śródrocznym skróconym'),
 (14.412745683990813, 'śrubokręta torx'),
 (14.412745683990813, 'środkowi mającemu'),
 (14.412745683990813, 'śmietanki owsianej'),
 (14.412745683990813, 'śmiertelnemu konfliktowi'),
 (14.412745683990813, 'śmierdzącego spoconego'),
 (14.412745683990813, 'śmiałość insynuującą'),
 (14.412745683990813, 'ślinka cieknie'),
 (14.412745683990813, 'ściśliwych piankowych'),
 (14.412745683990813, 'ściernych gąbek'),
 (14.412745683990813, 'łąki kośne'),
 (14.412745683990813, 'łyżwiarstwo figurowe'),
 (14.412745683990813, 'łukiem diamentowym'),
 (14.412745683990813, 'łopatką wieprzową'),
 (14.412745683990813, 'łaźnią olejową'),
 (14.412745683990813, 'łazikach marsjańskich'),
 (14.412745683990813, 'łatwopalnymi paczkami'),
 (14.412745683990813, 'łajdakami wykorzystującymi'),
 

Odfiltrowanie najrzadszych bigramów pozwala uzyskać bardziej rozsądne wyniki - większość naszych aktualnych wyników to charakterystyczne określenia składające się z dwóch słów (np. "klęska żywiołowa", "książeczka czekowa") lub nazwy własne (np. "Stucco Veneziano", "Bert Hellinger").

In [7]:
filter_rare = lambda element: bigrams_counter[element] < 5
discard_elements(bigrams_counter, filter_rare)

bigrams_pmi = [(pmi(bigram), bigram) for bigram in bigrams_counter]
top_bigrams_pmi = sorted(bigrams_pmi, reverse=True)[:50]
top_bigrams_pmi

[(15.656543516720237, 'stucco veneziano'),
 (15.656543516720237, 'królicza nora'),
 (15.656543516720237, 'klęska żywiołowa'),
 (15.656543516720237, 'bert hellinger'),
 (15.393509110886443, 'seair exim'),
 (15.393509110886443, 'sameer thakar'),
 (15.393509110886443, 'książeczka czekowa'),
 (15.393509110886443, 'gone fishin'),
 (15.393509110886443, 'electro plating'),
 (15.393509110886443, 'deming electro'),
 (15.171116689549995, 'wózkiem widłowym'),
 (15.171116689549995, 'stwardnienia rozsianego'),
 (15.171116689549995, 'stawu biodrowego'),
 (15.171116689549995, 'napędów taśmowych'),
 (15.171116689549995, 'kuala lumpur'),
 (15.171116689549995, 'króliczej nory'),
 (15.171116689549995, 'klatkę piersiową'),
 (15.171116689549995, 'billem gatesem'),
 (15.171116689549995, 'autot ldr'),
 (14.978471611607599, 'psychoterapeuta bert'),
 (14.978471611607599, 'obciążeniami zwrotnymi'),
 (14.978471611607599, 'kushagra nayan'),
 (14.978471611607599, 'konter pulsa'),
 (14.978471611607599, 'gałki oczne

## Zadanie 6

Na wyniki patrzymy teraz z innej perspektywy - chcemy zobaczyć, jak najczęstsze segmenty rozkładają się w ujęciu gramatycznym.

In [8]:
nlp = spacy.load('pl_core_news_sm')

dataset = load_dataset('clarin-knext/fiqa-pl', 'corpus')['corpus']['text']
dataset = map(nlp, dataset)
dataset = list(map(lambda word: f'{word.lemma_}:{word.tag_}'.lower(), chain.from_iterable(dataset)))

for i, word in enumerate(dataset[:10], 1):
    print(f'{i}. {word}')

Found cached dataset fiqa-pl (C:/Users/AGH/.cache/huggingface/datasets/clarin-knext___fiqa-pl/corpus/0.0.0/bada00640881ee3fd04c3b88df9edd435616d17e0a46faf05e63063858742140)


  0%|          | 0/1 [00:00<?, ?it/s]

1. nie:qub
2. mówić:fin
3. ,:interp
4. że:comp
5. nie:qub
6. podobać:fin
7. ja:ppron12
8. się:qub
9. też:qub
10. pomysł:subst


Ranking top-10 wartości PMI obliczony na korpusie po lematyzacji oraz tagowaniu daje bardzo podobne wyniiki - pojawiają się w nim te same lub należące do tych samych kategorii bigramy.

In [9]:
bigrams = [a + ' ' + b for a, b in zip(dataset, dataset[1:])]
bigrams = list(filter(lambda x: len(x.split(' ')) == 2, bigrams))
bigrams_counter = Counter(bigrams)
tokens_counter = Counter(dataset)

def filter_not_letters(element):
    a, b = element.split(' ')
    a, b = a.split(':')[0], b.split(':')[0]
    return any([c not in all_chars for c in a + b])

discard_elements(bigrams_counter, filter_not_letters)
discard_elements(bigrams_counter, filter_rare)

for i, (key, val) in zip(range(1, 50), bigrams_counter.items()):
    print(f'{i}. {key}: {val}')

print()

bigrams_pmi = [(pmi(bigram), bigram) for bigram in bigrams_counter]
top_bigrams_pmi_tag_lem = sorted(bigrams_pmi, reverse=True)[:10]
top_bigrams_pmi_tag_lem

1. nie:qub mówić:fin: 529
2. że:comp nie:qub: 4901
3. nie:qub podobać:fin: 155
4. podobać:fin ja:ppron12: 246
5. ja:ppron12 się:qub: 1407
6. się:qub też:qub: 97
7. szkolenie:ger w:prep: 5
8. w:prep miejsce:subst: 313
9. miejsce:subst praca:subst: 1080
10. ale:conj nie:qub: 4228
11. nie:qub móc:fin: 6580
12. móc:fin oczekiwać:inf: 88
13. że:comp firma:subst: 735
14. firma:subst to:pred: 92
15. to:pred zrobić:subst: 150
16. pracownik:subst to:pred: 17
17. to:pred nie:qub: 2701
18. nie:qub on:ppron3: 89
19. on:ppron3 praca:subst: 145
20. on:ppron3 tworzyć:fin: 12
21. być:inf może:fin: 1342
22. system:subst edukacyjny:adj: 9
23. edukacyjny:adj w:prep: 9
24. w:prep stany:subst: 921
25. stany:subst zjednoczone:adj: 1951
26. lub:conj on:ppron3: 203
27. on:ppron3 student:subst: 5
28. powinien:winien trochę:adv: 6
29. martwić:inf się:qub: 112
30. się:qub o:prep: 1727
31. umiejętność:subst rynkowy:adj: 6
32. rynkowy:adj w:prep: 91
33. w:prep zamian:burk: 422
34. zamian:burk za:prep: 330
35. za:p

[(15.89580735073129, 'emiratów:subst arabskich:subst'),
 (15.89580735073129, 'bert:subst hellinger:subst'),
 (15.632772944897496, 'seair:subst exim:subst'),
 (15.632772944897496, 'sameer:subst thakar:subst'),
 (15.632772944897496, 'gone:subst fishin:subst'),
 (15.632772944897496, 'electro:subst plating:subst'),
 (15.632772944897496, 'deming:xxx electro:subst'),
 (15.410380523561049, 'autot:subst ldr:subst'),
 (15.217735445618652, 'nawiasach:subst kwadratowych:adj'),
 (15.217735445618652, 'kushagra:subst nayan:subst')]

In [10]:
groups = defaultdict(list)

for bigram, count in bigrams_counter.items():
    a, b = bigram.split(' ')
    a, b = a.split(':')[1], b.split(':')[1]
    groups[(a, b)].append((count, bigram))

for i, key in zip(range(1, 11), groups):
    print(f'{i}. {key}')

1. ('qub', 'fin')
2. ('comp', 'qub')
3. ('fin', 'ppron12')
4. ('ppron12', 'qub')
5. ('qub', 'qub')
6. ('ger', 'prep')
7. ('prep', 'subst')
8. ('subst', 'subst')
9. ('conj', 'qub')
10. ('fin', 'inf')


## Podsumowanie

Kategorie najczęściej spotykanych bigramów są dość intuicyjne, jeśli chodzi o język naturalny, wyniki są zgodne z regułami łączenia słów w języku polskim:

- przyimek + rzeczownik (np. "na przykład"),
- przymiotnik + rzeczownik (np. "druga strona"),
- rzeczownik + przyimek (np. "podatek od"),
- rzeczownik + rzeczownik (np. "miejsce pracy"),
- przyimek + przymiotnik (np. "w którym"),
- rzeczownik + przymiotnik (np. "karta kredytowa"),
- rzeczownik + czasownik (np. "to jest"),
- przeczenie + czasownik (np. "nie jest"),
- czasownik + rzeczownik (np. "oznacza to"),
- rzeczownik + spójnik (np. "pieniądz i").

In [ ]:
groups_count = [(sum(map(lambda g: g[0], val)), key) for key, val in groups.items()]
groups_count = sorted(groups_count, reverse=True)
groups_count[:10]

In [ ]:
for _, group in groups_count[:10]:
    print('-' * 50)
    print(group)
    
    for i, v in enumerate(sorted(groups[group], reverse=True)[:5], 1):
        print(f'{i}. [{v[0]}] {v[1]}')
        
print('-' * 50)